In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import socket
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
from scipy import stats
from typing import Union, List
from futils import parse_version, snapshot
from hscpy.figures import PlotOptions
from hscpy.figures import abc as abc_fig
from hscpy.figures import sfs as sfs_fig
from hscpy import realisation, mitchell, abc, variant

PATH2BIN = Path("~").expanduser() / "hsc/target/release"
assert PATH2BIN.is_dir()

In [ ]:
%%bash -s "$PATH2BIN" --out version
$1/hsc --version

In [ ]:
USE_SCRATCH = True
SAVEFIG = True
SHOW_PRIORS = True
LATEST = True
# set to true if it's the first time running this nb: this
# will remove the runs that didn't finish running up to the
# last timepoint
FIRST_TIME = False

# the higher, the less precise and thus the more runs
# we set to these values because we aim to keep approx X runs
# PROPORTION_RUNS_DISCARDED, QUANTILE_SFS, QUANTILE_CLONES = 0.2, 0.25, 0.4
PROPORTION_RUNS_DISCARDED, QUANTILE_SFS, QUANTILE_CLONES = 0.2, 0.35, 0.35

options = PlotOptions(figsize=(7, 6), extension="svg", save=SAVEFIG)

In [ ]:
if LATEST:
    VERSION = parse_version(version)
else:
    VERSION = "v2.2.4"
PATH2SAVE = Path(f"./{VERSION}")

print("Running hsc with version:", VERSION)

if USE_SCRATCH:
    PATH2SIMS = Path("/data/scratch")
else:
    PATH2SIMS = Path("/data/home")
PATH2SIMS /= f"hfx923/hsc-draft/{VERSION}"

if socket.gethostname() == "5X9ZYD3":
    PATH2MITCHELL = Path("/mnt/c/Users/terenz01/Documents/SwitchDrive/PhD/hsc")
elif socket.gethostname() == "LAPTOP-CEKCHJ4C":
    PATH2MITCHELL = Path("/mnt/c/Users/fra_t/Documents/PhD/hsc")
else:
    PATH2MITCHELL = Path("~").expanduser()

In [ ]:
# 1. we dont use summary `Summary_cut.csv` because
#    it contains errors (i.e. duplicated and missing entries)
# 2. we dont use donor CB001 because we already have CB002 at age 0
# 3. we dont use donor KX007 because the genotype matrix is wrong,
#    they have uploded twice the same donor
donors = pd.DataFrame.from_records(
    [
        {"name": "CB002", "age": 0, "cells": 390, "clones": 0},
        {"name": "KX001", "age": 29, "cells": 407, "clones": 0},
        {"name": "KX002", "age": 38, "cells": 380, "clones": 1},
        {"name": "SX001", "age": 48, "cells": 362, "clones": 0},
        {"name": "AX001", "age": 63, "cells": 361, "clones": 1},
        {"name": "KX008", "age": 76, "cells": 367, "clones": 12},
        {"name": "KX004", "age": 77, "cells": 451, "clones": 15},
        {"name": "KX003", "age": 81, "cells": 328, "clones": 13},
    ]
)
donors

In [ ]:
# bins for abc
bins_mu = np.arange(0, 31, 1)
bins_tau = np.arange(0, 10.5, 0.5)
bins_s = np.arange(0, 0.21, 0.01)
bins_std = np.arange(0, 0.052, 0.002)

# Load simualted data both SFS and variant fractions

## Remove runs
Remove all the runs that didn't finish running. This is required because we load runs in a numpy array with fixed size.

In [ ]:
def files2remove(years: List[int], cells: List[int]) -> List[str]:
    assert len(years) == len(cells)

    paths = [
        Path(f"{PATH2SIMS}/{cell}cells/sfs/{year}dot0years")
        for year, cell in zip(years, cells)
    ]

    files = [file.stem for path in paths for file in path.iterdir()]
    files2remove = {
        file for file in set(files) if files.count(file) < len(donors.age.tolist())
    }
    print(f"{len(files2remove)} files to remove")
    return files2remove

In [ ]:
%%time
if FIRST_TIME:
    runs2remove = files2remove(
        years=donors.age.tolist(),
        cells=donors.cells.tolist(),
    )
    print(f"found {len(runs2remove)} runs to remove")

    for r in donors[["age", "cells"]].itertuples():
        print(r)
        for f in runs2remove:
            myf = f"{PATH2SIMS}/{r.cells}cells/sfs/{r.age}dot0years/{f}"
            try:
                Path(myf).with_suffix(".json").unlink()
            except FileNotFoundError:
                pass
            try:
                Path(myf.replace("sfs", "variant_fraction")).with_suffix(
                    ".csv"
                ).unlink()
            except FileNotFoundError:
                pass

## Load data

In [ ]:
%%time
sfs_sims = dict()
counts_sims = dict()

for r in donors[["name", "age", "cells"]].itertuples():
    print(f"\tloading sims SFS for donor {r.name} with {r.cells} cells")
    path2sfs = Path(PATH2SIMS / f"{r.cells}cells/sfs/")
    sfs_sims.update(realisation.load_all_sfs_by_age(path2sfs))

    print(f"\tloading sims variant counts for donor {r.name} with {r.cells} cells")
    counts_sims.update(
        variant.load_all_detected_var_counts_by_age(
            PATH2SIMS / f"{r.cells}cells/variant_fraction", 0.01
        )
    )

In [ ]:
counts = variant.variant_counts_detected_df(counts_sims)
fig, ax = plt.subplots(1, 1)
sns.lineplot(
    counts,
    x="age",
    y="variant counts detected",
    errorbar=lambda x: (np.min(x), np.max(x)),
    ax=ax,
    label="min-max",
)
sns.lineplot(
    counts,
    x="age",
    y="variant counts detected",
    errorbar="sd",
    ax=ax,
    color="orange",
    label="std",
)
ax.legend()
plt.show()
print(counts[["variant counts detected", "age"]].groupby("age").describe())

# Run ABC on the real data

## Load the data from Mitchell's paper
We have excluded two donors from the ABC:
1. exclude KX007 bc they have uploded twice the same donor
3. exclude CB001 bc it maps to to the same timepoint as CB002 (same age 0)

In [ ]:
%%time
target_sfs = {
    r.age: mitchell.sfs_donor_mitchell(
        r.name, r.age, PATH2MITCHELL, remove_indels=False
    )
    for r in donors[["name", "age"]].itertuples()
}
assert [
    ele[2] for ele in target_sfs.values()
] == donors.cells.tolist(), (
    "cells found in genotype matrices do not match the extected nb of cells"
)

## Run ABC on subsampled simulated data

### Compute the summary statistics 
1. wasserstein metric
2. the number of clones
3. the KS stat (not implemented yet)

After having computed the statistics from the simulations and Mitchell's data, we won't use the data anymore but just the summary statistic dataframe `abc_mitchell`: we are going to filter (and keep) the runs by selecting from this dataframe.

In [ ]:
print("wasserstein metric")
abc_mitchell = abc.sfs_summary_statistic_wasserstein(
    sfs_sims,
    {
        k: ele[-1] for k, ele in target_sfs.items()
    },  # ele[-1] means the SFS, k is the age
    "mitchell",
)

# add information about clones from Mitchell's fig 5a
print("clones metric")
abc_mitchell = abc_mitchell.merge(
    right=counts[["age", "idx", "variant counts detected"]],
    how="left",
    left_on=["idx", "timepoint"],
    right_on=["idx", "age"],
    validate="one_to_one",
)
abc_mitchell.dropna(inplace=True)
assert (
    not abc_mitchell.isna().any().any()
), "cannot match the nb of clones data to the abc results"
abc_mitchell = donors.merge(
    right=abc_mitchell, how="right", on="age", validate="one_to_many"
)
# 2. number of clones
abc_mitchell["clones diff"] = (
    abc_mitchell["clones"] - abc_mitchell["variant counts detected"]
).abs()
abc_mitchell = abc.summary_statistic_relative_diff_clones(abc_mitchell)

abc_mitchell.shape

### Show priors

In [ ]:
# TODO priors are now s /tau and sigma/ tau not s and sigma
if SHOW_PRIORS:
    priors = abc_mitchell[["mu", "u", "s", "std", "tau"]].drop_duplicates()
    priors["s/tau"] = priors["s"] / priors["tau"]

    fig, ax = plt.subplots(1, 1, figsize=[7, 6])
    ax = abc_fig.plot_prior(priors["s/tau"], ax=ax, binwidth=0.01)
    plt.show()

    fig, ax = plt.subplots(1, 1, figsize=[7, 6])
    priors["std/tau"] = priors["std"] / priors["tau"]
    ax = abc_fig.plot_prior(priors["std/tau"], ax=ax, binwidth=0.001)
    plt.show()

    fig, ax = plt.subplots(1, 1, figsize=[7, 6])
    ax = abc_fig.plot_prior(priors["tau"], ax=ax, binwidth=0.1)
    plt.show()

    fig, ax = plt.subplots(1, 1, figsize=[7, 6])
    ax = abc_fig.plot_prior(priors["mu"], ax=ax, discrete=False)
    plt.show()

    fig, ax = plt.subplots(1, 1, figsize=[7, 6])
    ax = abc_fig.plot_prior(priors["u"], ax=ax)
    plt.show()

    fig, ax = plt.subplots(1, 1, figsize=[7, 6])
    sns.histplot(abc_mitchell["wasserstein"], binwidth=0.01, ax=ax)
    plt.show()

    sns.pairplot(priors[["s/tau", "std/tau", "tau", "mu"]], kind="hist")
    if options.save:
        plt.savefig(f"priors.{options.extension}")
    plt.show()

### Run abc considering all timepoints at the same time

In [ ]:
def find_map_by_cut(
    view: pd.Series, bins: List[Union[float, int]]
) -> Union[float, None]:
    max_a_posteriori = pd.cut(view, bins=bins).mode()
    if max_a_posteriori.shape[0] > 1:
        print(
            f"cannot compute MAP because more than one mode have been found {max_a_posteriori}"
        )
        return None
    max_a_posteriori = max_a_posteriori.iloc[0]
    return max_a_posteriori.mid

In [ ]:
# plot all posteriors for all timepoints separately, for all metrics
for t in sorted(target_sfs.keys()):
    # for metric in ("rel clones diff", ):
    # for metric in ("clones diff", ):
    # for metric in ("wasserstein", "rel clones diff"):
    for metric in ("wasserstein",):
        fig, axes = plt.subplots(2, 2, layout="tight", sharey=True)
        print(f"Running ABC on {metric} metric at age {t}")
        idx = abc.run_abc_per_single_timepoint(
            abc_mitchell,
            t,
            quantile=0.01,
            metric=metric,
        ).get_idx()
        print(len(idx))
        view = abc_mitchell[abc_mitchell.idx.isin(idx)]

        for i, (theta, lim) in enumerate(
            zip(
                ["s", "std", "mu", "tau"],
                [
                    (bins_s[0], bins_s[-1]),
                    (bins_std[0], bins_std[-1]),
                    (bins_mu[0], bins_mu[-1]),
                    (bins_tau[0], bins_tau[-1]),
                ],
            )
        ):
            ax = axes[np.unravel_index(i, (2, 2))]
            sns.histplot(view[theta], ax=ax)  # stat="percent")
            ax.set_xlim(lim)
        fig.suptitle(f"{t} years with {metric}", y=0.9, fontsize="small")
        plt.show()

In [ ]:
runs2keep = abc.run_abc_sfs_clones(
    abc_mitchell, QUANTILE_SFS, QUANTILE_CLONES, PROPORTION_RUNS_DISCARDED
)
view = abc_mitchell[abc_mitchell.idx.isin(runs2keep)].drop_duplicates(subset="idx")

view["s_tau"] = view.s / view.tau
view["std_tau"] = view["std"] / view.tau

assert not view.empty, "empty posterior"
print(
    f"ABC combined kept {len(runs2keep)} runs over a total of {abc_mitchell.idx.unique().shape[0]} runs"
)

# plots
map_mu = find_map_by_cut(view.mu, range(0, 21))
# map_s = find_map_by_cut(view.s, np.arange(0, 0.45, kwargs_s["binwidth"]).tolist())
# map_std = find_map_by_cut(view["std"], np.arange(0, 0.15, kwargs_std["binwidth"]).tolist())

gs = []
gs.append(abc_fig.plot_results(view, "s_tau", "mu", bins_s, bins_mu))
plt.savefig(
    f"posterior{len(gs)}.{options.extension}", bbox_inches="tight", pad_inches=0
)
plt.show()

gs.append(abc_fig.plot_results(view, "s_tau", "std_tau", bins_s, bins_std))
plt.savefig(
    f"posterior{len(gs)}.{options.extension}", bbox_inches="tight", pad_inches=0
)
plt.show()

gs.append(abc_fig.plot_results(view, "s_tau", "tau", bins_s, bins_tau))
plt.savefig(
    f"posterior{len(gs)}.{options.extension}", bbox_inches="tight", pad_inches=0
)
plt.show()

gs.append(abc_fig.plot_results(view, "mu", "tau", bins_mu, bins_tau))
plt.savefig(
    f"posterior{len(gs)}.{options.extension}", bbox_inches="tight", pad_inches=0
)
plt.show()

gs.append(abc_fig.plot_results(view, "mu", "std_tau", bins_mu, bins_std))
plt.savefig(
    f"posterior{len(gs)}.{options.extension}", bbox_inches="tight", pad_inches=0
)
plt.show()


gs.append(abc_fig.plot_results(view, "tau", "std_tau", bins_tau, bins_std))
plt.savefig(
    f"posterior{len(gs)}.{options.extension}", bbox_inches="tight", pad_inches=0
)
plt.show()

# TODO: maybe iterate over gs to remove the grid in the marginal plots?
verbose = False
idx2show = dict()

for t in sorted(abc_mitchell.timepoint.unique()):
    if verbose:
        title = f"age: {t} years, quantile threshold: {abc_mitchell.loc[abc_mitchell.timepoint == t, 'wasserstein'].quantile(quantile):.2f}"
    else:
        title = f"age: {round(t)} years"

    idx2show[t] = abc_fig.get_idx_smaller_distance_clones_idx(
        abc_mitchell[abc_mitchell.timepoint == t], runs2keep
    )

    fig = sfs_fig.plot_sfs_cdf(
        [idx2show[t]], target_sfs[t][3], sfs_sims[t], t, verbose=verbose, alpha=0.7
    )
    """fig.suptitle(
        title,
        x=0.4,
    )"""
    if options.save:
        fig.savefig(f"sfs_{t}years.{options.extension}")
    plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, layout="tight")
sns.lineplot(
    abc_mitchell[abc_mitchell.idx.isin(runs2keep)],
    x="age",
    y="variant counts detected",
    errorbar=lambda x: (np.min(x), np.max(x)),
    ax=ax,
    label="min-max",
)
sns.lineplot(
    abc_mitchell[abc_mitchell.idx.isin(runs2keep)],
    x="age",
    y="variant counts detected",
    errorbar="sd",
    ax=ax,
    color="orange",
    label="std",
)
sns.scatterplot(
    data=abc_mitchell[["age", "clones"]].drop_duplicates(),
    x="age",
    y="clones",
    marker="x",
    linewidths=2,
    color="purple",
    label="Mitchell",
)
ax.legend()
if options.save:
    fig.savefig(f"variants_abc.{options.extension}")
plt.show()

In [ ]:
ts = list(idx2show.keys())
mus, ss = (
    [
        abc_mitchell[(abc_mitchell.timepoint == t) & (abc_mitchell.idx == idx)].mu.iloc[
            0
        ]
        for t, idx in idx2show.items()
    ],
    [
        abc_mitchell[(abc_mitchell.timepoint == t) & (abc_mitchell.idx == idx)].s.iloc[
            0
        ]
        for t, idx in idx2show.items()
    ],
)
sns.histplot(mus, binwidth=0.5)
plt.show()
sns.histplot(ss, binwidth=0.005)
plt.show()

plt.plot(ts, mus, marker="x", mew=2)
plt.show()

plt.plot(ts, ss, marker="x", mew=2)
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, layout="constrained")
(view.s / view.tau).hist(ax=ax, bins=20)
ax.set_xlabel(r"$s/\tau$")
ax.set_ylabel("counts")
mean = (view.s / view.tau).mean()
ax.axvline(mean, color="black")
ax.annotate(
    f"mean = {mean:.3f}", xy=(0.5, 1.01), xycoords=("axes fraction", "axes fraction")
)
if options.save:
    fig.savefig(f"s_over_tau.{options.extension}")
plt.show()

fig, ax = plt.subplots(1, 1, layout="constrained")
(view["std"] / view.tau).hist(ax=ax, bins=20)
ax.set_xlabel(r"$\sigma/\tau$")
ax.set_ylabel("counts")
mean = (view["std"] / view.tau).mean()
ax.axvline(mean, color="black")
ax.annotate(
    f"mean = {mean:.3f}", xy=(0.5, 1.01), xycoords=("axes fraction", "axes fraction")
)

if options.save:
    fig.savefig(f"sigma_over_tau.{options.extension}")
plt.show()

In [ ]:
view[["s", "tau", "std"]].corr()

In [ ]:
# TODO remove?
s = np.asarray(view.s.tolist())
tau = np.asarray(view.tau.tolist())
A = np.vstack([s, np.ones(len(s))]).T
m, c = np.linalg.lstsq(A, tau, rcond=None)[0]

fig, ax = plt.subplots(1, 1)
ax.plot(s, tau, "o", label="Original data", markersize=10)
ax.plot(s, m * s + c, "r", label=f"Fit {m:.2f}+{c:.2f}")
ax.set_xlabel(r"$s$")
ax.set_ylabel(r"$\tau$")
ax.legend()
plt.show()

In [ ]:
def shape_scale_from_mean_std(mean, std):
    return mean**2 / std**2, std**2 / mean

In [ ]:
fig, ax = plt.subplots(1, 1, layout="constrained")
mean = view.tau.mean()
median = view.tau.median()
view.tau.hist(ax=ax, bins=15)
ax.axvline(mean, color="black", label=f"mean={mean:.2f}")
ax.axvline(mean, color="red", label=f"median={median:.2f}")
ax.legend()
ax.set_xlim([bins_tau[0], bins_tau[-1]])
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1)
view[["s_tau", "std_tau"]].drop_duplicates().plot(
    ax=ax, x="s_tau", y="std_tau", kind="scatter"
)
view.loc[
    (view.std_tau > 0.016) & (view.s_tau < 0.03), ["s_tau", "std_tau"]
].drop_duplicates().plot(ax=ax, x="s_tau", y="std_tau", kind="scatter", c="yellow")
ax.set_ylim([0, 0.05])
ax.set_xlim([0, 0.2])
ax.set_xlabel(r"$\eta$")
ax.set_ylabel(r"$\sigma$")
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1)
for row in view[["s_tau", "std_tau"]].drop_duplicates().itertuples():
    shape, scale = shape_scale_from_mean_std(row.s_tau, row.std_tau)
    c = "yellow" if row.std_tau > 0.016 and row.s_tau < 0.03 else "grey"
    # c = "blue"
    x = np.linspace(
        stats.gamma.ppf(0.01, shape, scale=scale),
        stats.gamma.ppf(0.99, shape, scale=scale),
        100,
    )
    ax.plot(x, stats.gamma.pdf(x, shape, 0, scale), c=c, lw=1, alpha=0.2)

shape, scale = shape_scale_from_mean_std(view.s_tau.mean(), view.std_tau.mean())
x = np.linspace(
    stats.gamma.ppf(0.01, shape, scale=scale),
    stats.gamma.ppf(0.99, shape, scale=scale),
    100,
)
ax.plot(x, stats.gamma.pdf(x, shape, 0, scale), c="red", lw=1, alpha=0.9)
ax.set_ylim([0, 150])
ax.set_xlim([0, 0.25])
ax.set_ylabel("pdf")
ax.set_xlabel(r"$s$")
plt.show()

In [ ]:
fabre_posteriors = pd.read_csv("abcAcceptedParams_Intermediate_5D_normedFitness.csv")
fabre_posteriors

In [ ]:
def plot_posterior_mitchell_fabre2(
    mitchell: pd.Series,
    fabre: pd.Series,
    xlabel: str,
    bins,
    ax,
    legend: bool = True,
    colors=None,
):
    ax.hist(
        fabre,
        align="mid",
        alpha=0.58,
        density=True,
        bins=bins,
        edgecolor="black",
        label="Fabre",
        color=colors[0] if colors else None,
    )
    ax.hist(
        mitchell,
        align="mid",
        alpha=0.58,
        density=True,
        bins=bins,
        edgecolor="black",
        label="Mitchell",
        color=colors[1] if colors else None,
    )

    ax.set_xlim([bins[0], bins[-1]])
    ax.set_xlabel(xlabel)
    ax.set_ylabel("pdf")
    if legend:
        ax.legend()
    return ax

In [ ]:
import matplotlib.gridspec as gridspec
from matplotlib import ticker
from matplotlib.patches import Patch


def fmt_two_digits(x, pos):
    return f"{x:.2f}"


from matplotlib.lines import Line2D

colors = ["#1b9e77", "#d95f02"]
custom_lines = [
    Patch(facecolor=colors[0], alpha=0.7, edgecolor="black", label="Fabre"),
    Patch(facecolor=colors[1], alpha=0.7, edgecolor="black", label="Mitchell"),
]

fig = plt.figure(figsize=[8, 7.5], layout="constrained")

gs0 = gridspec.GridSpec(2, 1, figure=fig, height_ratios=[1, 2])

# gammas and legend
gs00 = gridspec.GridSpecFromSubplotSpec(1, 4, subplot_spec=gs0[0])
ax1 = fig.add_subplot(gs00[0, :-1])
shape, scale = shape_scale_from_mean_std(
    fabre_posteriors.s.mean(), fabre_posteriors["σ"].mean()
)
x = np.linspace(
    stats.gamma.ppf(0.01, shape, scale=scale),
    stats.gamma.ppf(0.99, shape, scale=scale),
    100,
)
ax1.plot(x, stats.gamma.pdf(x, shape, 0, scale), label="Fabre", c=colors[0])
shape, scale = shape_scale_from_mean_std(view.s_tau.mean(), view.std_tau.mean())
x = np.linspace(
    stats.gamma.ppf(0.01, shape, scale=scale),
    stats.gamma.ppf(0.99, shape, scale=scale),
    100,
)
ax1.plot(x, stats.gamma.pdf(x, shape, 0, scale), label="Mitchell", c=colors[1])
ax1.set_ylim([-1, 20])
ax1.set_ylabel("pdf")
ax1.set_xlabel(r"$s$")  # TODO
ax2 = fig.add_subplot(gs00[0, -1])
ax2.axis("off")
ax2.legend(custom_lines, ["Fabre", "Mitchell"], frameon=False, fontsize=16)

# posteriors
gs01 = gs0[1].subgridspec(2, 2)

ax4 = fig.add_subplot(gs01[0, 0])
plot_posterior_mitchell_fabre2(
    view["s_tau"],
    fabre_posteriors["s"],
    r"$\eta$",
    bins_s,
    ax4,
    legend=False,
    colors=colors,
)
ax4.set_xlim([-0.02, 0.21])
ax4.axvline(x=fabre_posteriors["s"].mean(), mew=3, ls="--", c=colors[0])
ax4.axvline(x=view["s_tau"].mean(), mew=3, ls="--", c=colors[1])


ax5 = fig.add_subplot(gs01[0, 1])
plot_posterior_mitchell_fabre2(
    view["std_tau"],
    fabre_posteriors["σ"],
    r"$\sigma$",
    bins_std,
    ax5,
    legend=False,
    colors=colors,
)
ax5.set_xlim([-0.005, 0.055])
ax5.axvline(x=fabre_posteriors["σ"].mean(), mew=3, ls="--", c=colors[0])
ax5.axvline(x=view["std_tau"].mean(), mew=3, ls="--", c=colors[1])
ax5.xaxis.set_major_formatter(ticker.FuncFormatter(fmt_two_digits))

ax6 = fig.add_subplot(gs01[1, 0])
plot_posterior_mitchell_fabre2(
    view["tau"],
    fabre_posteriors["τ"],
    r"$\tau$",
    bins_tau,
    ax6,
    legend=False,
    colors=colors,
)
ax6.set_xlim([-0.5, 10.5])
ax6.axvline(x=fabre_posteriors["τ"].mean(), mew=3, ls="--", c=colors[0])
ax6.axvline(x=view["tau"].mean(), mew=3, ls="--", c=colors[1])

ax7 = fig.add_subplot(gs01[1, 1])
plot_posterior_mitchell_fabre2(
    view["mu"],
    fabre_posteriors["μ"],
    r"$\mu$",
    bins_mu,
    ax7,
    legend=False,
    colors=colors,
)
ax7.set_xlim([-0.5, 31])
ax7.axvline(x=fabre_posteriors["μ"].mean(), mew=3, ls="--", c=colors[0])
ax7.axvline(x=view["mu"].mean(), mew=3, ls="--", c=colors[1])

if options.save:
    fig.savefig(f"figure4.{options.extension}", transparent=True)

plt.show()

In [ ]:
plot_posterior_mitchell_fabre2(view["s_tau"], fabre_posteriors["s"], r"$\eta$", bins_s)
plt.show()

In [ ]:
plot_posterior_mitchell_fabre2(
    view["std_tau"], fabre_posteriors["σ"], r"$\sigma$", bins_std, legend=False
)
plt.show()

In [ ]:
plot_posterior_mitchell_fabre2(view["mu"], fabre_posteriors["μ"], r"$\mu$", bins_mu)
plt.show()

In [ ]:
plot_posterior_mitchell_fabre2(view["tau"], fabre_posteriors["τ"], r"$\tau$", bins_tau)
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1)
shape, scale = shape_scale_from_mean_std(
    fabre_posteriors.s.mean(), fabre_posteriors["σ"].mean()
)
x = np.linspace(
    stats.gamma.ppf(0.01, shape, scale=scale),
    stats.gamma.ppf(0.99, shape, scale=scale),
    100,
)
ax.plot(x, stats.gamma.pdf(x, shape, 0, scale), label="Fabre")

shape, scale = shape_scale_from_mean_std(view.s_tau.mean(), view.std_tau.mean())
x = np.linspace(
    stats.gamma.ppf(0.01, shape, scale=scale),
    stats.gamma.ppf(0.99, shape, scale=scale),
    100,
)
ax.plot(x, stats.gamma.pdf(x, shape, 0, scale), label="Mitchell")

# ax.set_xlim([-0.1, 0.25])
ax.set_ylabel("pdf")
ax.set_xlabel(r"$s$")  # TODO
ax.legend()
plt.show()

## Run ABC on subsampled simulated data

In [ ]:
# TODO work with s/tau not s
validation_idx3 = abc_fig.SyntheticValidation(3, sfs_sims, counts)
gs02_02 = validation_idx3.compute_posteriors(0.2, 0.2)
gs03_03 = validation_idx3.compute_posteriors(0.3, 0.3)

In [ ]:
validation_idx2 = abc_fig.SyntheticValidation(2, sfs_sims, counts)
gs02_02 = validation_idx2.compute_posteriors(0.2, 0.2)
gs03_03 = validation_idx2.compute_posteriors(0.3, 0.3)